**Histórico dos dados dos fundos de investimentos**
* https://dados.cvm.gov.br/dataset/fi-doc-cda
* https://dados.cvm.gov.br/dataset/fi-doc-inf_diario

* Observação: muitas vezes, os arquivos excel do site acima não se atualizam rapidamente. Para ver a posição mais atualizada entrar no site:
* https://conteudo.cvm.gov.br/menu/regulados/fundos/consultas/fundos.html
---
O arquivo zipado 'cda_fi' contém vários arquivos excel. Os arquivos que eu tenho interesse são:

* cda_fi_BLC_1: arquivo que contém os 'Títulos Públicos';
* cda_fi_BLC_2: arquivo que contém as 'Cotas de Fundos';
* cda_fi_BLC_4: arquivo que contém as 'Ações', 'BDRs', 'Opções', 'Mercado futuro', 'Debêntures', 'Obrigações por ações' (posição vendida);
* cda_fi_BLC_7: arquivo que contém os 'Investimentos no Exterior';
* cda_fi_BLC_8: arquivo que contém os 'Investimentos no Exterior';
* cda_fi_PL: arquivo que contém o PL dos fundos de investimentos.

---
O arquivo zipado 'inf_diario_fi' contém apenas um arquivo excel.

In [1]:
import glob
import numpy as np
import os
import pandas as pd
from pandas import DataFrame
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import zipfile

## Extraindo os arquivos zipados

### Extração dos arquivos "cda_fi"

In [ ]:
# Mês e ano escolhido
mes = '10'
ano = '2023'

# Nome do arquivo zipado
zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm_zip//cda_fi_{ano}{mes}.zip'

# # Visualizando a lista dos nomes dos arquivos dentro do arquivo zipado
# with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
#     print("Arquivos dentro do arquivo zipado:")
#     print(zip_ref.namelist())

# Lista dos nomes dos arquivos que você deseja extrair
arquivos_a_extrair = [
    f'cda_fi_BLC_1_{ano}{mes}.csv', 
    f'cda_fi_BLC_2_{ano}{mes}.csv', 
    f'cda_fi_BLC_4_{ano}{mes}.csv', 
    f'cda_fi_BLC_7_{ano}{mes}.csv', 
    f'cda_fi_BLC_8_{ano}{mes}.csv',
    f'cda_fi_PL_{ano}{mes}.csv'
]

# Caminho do diretório do diretório do arquivo não zipado
destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    # Extraindo os arquivos selecionados para um diretório específico
    for arquivo in arquivos_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        # Caminhos dos arquivos em formato csv e parquet
        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Lendo os arquivos csv
        df = pd.read_csv(csv_path, sep=';', encoding='ISO-8859-1', low_memory=False)

        # Transformando esses arquivos em parquet
        df.to_parquet(parquet_path)

# Entrando para o diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

# Loop para deletar cada arquivo csv encontrado
for arquivo in arquivos_csv:
    os.remove(arquivo)

### Extração dos arquivos "inf_diario_fi"

In [ ]:
# Descompactando apenas um arquivo

# Mês e ano escolhido
mes = '11'
ano = '2023'

# Nome do arquivo zipado
zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm_zip//inf_diario_fi_{ano}{mes}.zip'

# Lista dos nomes dos arquivos que você deseja extrair
arquivo_a_extrair = [f'inf_diario_fi_{ano}{mes}.csv']

# Caminho do diretório do diretório do arquivo não zipado
destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    # Extraindo os arquivos selecionados para um diretório específico
    for arquivo in arquivo_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        # Caminhos dos arquivos em formato csv e parquet
        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Lendo os arquivos csv
        df = pd.read_csv(csv_path, sep=';', encoding='ISO-8859-1', low_memory=False)

        # Transformando esses arquivos em parquet
        df.to_parquet(parquet_path)

# Entrando para o diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

# Loop para deletar cada arquivo csv encontrado
for arquivo in arquivos_csv:
    os.remove(arquivo)

In [ ]:
# Descompactando vários arquivos
for _ in ['01', '02', '03', '04']: 
    
    # Ano escolhido
    ano = '2024'

    # Nome do arquivo zipado
    zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm_zip//inf_diario_fi_{ano}{_}.zip'

    # Lista dos nomes dos arquivos que você deseja extrair
    arquivo_a_extrair = [f'inf_diario_fi_{ano}{_}.csv']

    # Caminho do diretório do diretório do arquivo não zipado
    destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm'

    # Abrindo o arquivo zipado
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        # Extraindo os arquivos selecionados para um diretório específico
        for arquivo in arquivo_a_extrair:
            zip_ref.extract(arquivo, destino_dir)

            # Caminhos dos arquivos em formato csv e parquet
            csv_path = f'{destino_dir}//{arquivo}'
            parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

            # Lendo os arquivos csv
            df = pd.read_csv(csv_path, sep=';', encoding='ISO-8859-1', low_memory=False)

            # Transformando esses arquivos em parquet
            df.to_parquet(parquet_path)

    # Entrando para o diretório especificado
    os.chdir(destino_dir)

    # Encontrando todos os arquivos .csv no diretório
    arquivos_csv = glob.glob('*.csv')

    # Loop para deletar cada arquivo csv encontrado
    for arquivo in arquivos_csv:
        os.remove(arquivo)

## Funções

In [2]:
def rentabilidade_fundo(df: pd.DataFrame, cnpj: str, nome_fundo: str):
    """
    Calcula a rentabilidade mensal e anual do fundo selecionado.

    Parameters:
    df: Dataframe que contem as cotas dos fundos.
    cnpj: cnpj do fundo de investimento que você está procurando.
    nome_fundo: nome do fundo.

    Returns:
    df_ret_mensal: Dataframe dos retornos mensais.
    df_ret_anual: Dataframe dos retornos anuais.

    NOTE:Para calcular a rentabilidade mensal dos fundos - o site MaisRetorno utiliza última cota do mês anterior com a última cota do mês seguinte.
    Para calcular a rentabilidade anual dos fundos - o site MaisRetorno utiliza última cota do ano anterior com a última cota do ano seguinte.
    Por isso, eu tive que baixar a cota do mês 12/2022.
    """
    # Selecionando o fundo de investimentos específicos
    filt_cnpj = (df['CNPJ_FUNDO'] == cnpj)
    fundo_espec = df.loc[filt_cnpj]

    # Selecionando os últimos dias de cada mês
    last_days = fundo_espec.groupby(fundo_espec.index.to_period('M')).tail(1)
    # Selecionando apenas a coluna 'VL_QUOTA'
    last_days = last_days.loc[:, 'VL_QUOTA']
    # Calculando a rentabilidade do mês
    ret_mensal = round(last_days.pct_change()*100, 2)
    # Como eu estou utilizando a última cota do mês anterior com a a última cota do mês seguinte, a 1º rentabilidade ('2022-12-30') dessa série irá ser um NaN
    ret_mensal = ret_mensal.dropna()
    # Criando o df p/ os retornos mensais
    df_ret_mensal = pd.DataFrame(ret_mensal)
    # Renomeando a coluna 'VL_QUOTA'
    df_ret_mensal = df_ret_mensal.rename(columns={f'VL_QUOTA':'ret'})

    # Transformando as poncentagens em taxa unitária
    df_ret_mensal['taxa_unit'] = 1 + (df_ret_mensal[f'ret'] / 100) 
    # Lista dos anos
    lst_years = df_ret_mensal.index.year.unique()
    # Transformando em string
    lst_years = lst_years.astype(str)
    # Calculando o retorno anual - aculumando as porcentagens de cada ano
    lst_ret_anual = []
    for _ in lst_years:
        ret_anual = round((df_ret_mensal.loc[_, 'taxa_unit'].agg(lambda x : x.prod()) -1) * 100, 2)
        lst_ret_anual.append(ret_anual)
    # Criando o df p/ os retornos anuais
    df_ret_anual = pd.DataFrame(lst_ret_anual, index=lst_years, columns=['ret'])
    # Adicionando o nome do fundo na string 
    df_ret_anual = df_ret_anual.rename(columns={'ret': f'ret_{nome_fundo}'})
    # Transformando o index em datetime
    df_ret_anual.index = pd.to_datetime(df_ret_anual.index)
    # Transformando o formato do index ('ano') p/ conseguir concatenar com as rentabilidades dos fundos
    df_ret_anual.index = df_ret_anual.index.to_period('Y')

    # Para não mostrar a coluna 'taxa_unit', selecionando apenas a coluna 'ret_mensal'
    df_ret_mensal_final = df_ret_mensal[[f'ret']]
    # Transformando o formato do index para 'ano-mes'
    df_ret_mensal_final.index = df_ret_mensal.index.to_period('M')
    # Adicionando o nome do fundo na string 
    df_ret_mensal_final = df_ret_mensal_final.rename(columns={'ret': f'ret_{nome_fundo}'})

    return df_ret_mensal_final, df_ret_anual


def rentabilidade_fundo_benchmark(
        df_fundo: pd.DataFrame, 
        df_benchmark: pd.DataFrame, 
        nome_fundo: str,
        nome_benchmark: str
    ) -> pd.DataFrame:
    """
    Parameters:
    df_fundo: Dataframe de rentabilidade do fundo.
    df_benchmark: Dataframe de rentabilidade do benchmark.
    nome_fundo: nome do fundo.
    nome_benchmark: nome do benchmark.

    Returns:
    Dataframe que contém a rentabilidade do fundo e do benchmark.
    """
    # Juntando com o df da rentabilidade mensal com o df de rentabilidade do benchmark
    df_ret_fundo_benchmark = pd.concat([df_fundo, df_benchmark], axis=1)

    # Calculando a performance do fundo - ele conseguiu bater o benchmark?
    df_ret_fundo_benchmark['performance'] = df_ret_fundo_benchmark[f'ret_{nome_fundo}'] - df_ret_fundo_benchmark[f'ret_{nome_benchmark}']

    return df_ret_fundo_benchmark


def open_cda_1(path: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_1'.

    Parameters:
    path: caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_1'
    """
    # Lendo o arquivo
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'TP_TITPUB', 'DT_VENC']]

    # Mesclando as colunas 'TP_TITPUB' e 'DT_VENC' em apenas em uma coluna
    df['TP_TITPUB'] = df['TP_TITPUB'] + ' ' + df['DT_VENC']

    # Removendo a coluna 'DT_VENC'
    df = df.drop('DT_VENC', axis=1)

    # Renomeando a coluna 'TP_TITPUB' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_2/4/7/8 para fazer depois juntar os dfs
    df = df.rename(columns={'TP_TITPUB':'CD_ATIVO'})

    # Transformando os dtypes das colunas.
    df['TP_FUNDO'] = df['TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df['DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df['TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df['TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df['VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df['CD_ATIVO'].astype(str)

    return df


def open_cda_2(path: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_2'.

    Parameters:
    path: caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_2'
    """
    # Lendo o arquivo. Adicionei o 'low_memory=False' para não dar o aviso -> DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'NM_FUNDO_COTA']]

    # Renomeando a coluna 'NM_FUNDO_COTA' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4/7/8 para fazer depois juntar os dfs
    df = df.rename(columns={'NM_FUNDO_COTA':'CD_ATIVO'})

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df['TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df['DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df['TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df['TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df['VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df['CD_ATIVO'].astype(str)

    return df


def open_cda_4(path: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_4'.

    Parameters:
    path: caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_4'
    """
    # Lendo o arquivo
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'CD_ATIVO']]

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    return df


def open_cda_7(path: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_7'.

    Parameters:
    path: caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_7'
    """
    # Lendo o arquivo
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas.
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'EMISSOR']]

    # Renomeando a coluna 'EMISSOR' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4 p/ fazer depois juntar os dfs.
    df.rename(columns={"EMISSOR": "CD_ATIVO"}, inplace=True)

    # Transformando os dtypes das colunas.
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    return df


def open_cda_8(path: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_8'.

    Parameters:
    path: caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_8'
    """
    # Lendo o arquivo
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'DS_ATIVO']]

    # Renomeando a coluna 'DS_ATIVO' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4 p/ fazer depois juntar os dfs
    df.rename(columns={"DS_ATIVO": "CD_ATIVO"}, inplace=True)

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    # Selecionando apenas o ativo 'BDR', porque neste arquivo também possui um ativo chamado 'Títulos Públicos', mas não é o principal 'Títulos Públicos', que está no 'cda_fi_BLC_1'
    filt_bdr = (df['TP_APLIC'] == 'Brazilian Depository Receipt - BDR')
    df = df.loc[filt_bdr].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    
    return df


def pl_fundo(path: str, cnpj: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_PL'.
    
    Paramenters:
    path: caminho do arquivo.
    cnpj: cnpj do fundo de investimento que você está procurando.

    Returns:
    Dataframe com o valor do patrimônio líquido do fundo de investimentos específico.
    """
    # Lendo o arquivo
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando o fundo de investimentos específicos
    filt_cnpj = df['CNPJ_FUNDO'] == cnpj
    fundo_espec = df.loc[filt_cnpj]

    # # Transformando os dtypes da coluna
    # fundo_espec['VL_PATRIM_LIQ'] = fundo_espec.loc[:, 'VL_PATRIM_LIQ'].astype(float)

    return fundo_espec['VL_PATRIM_LIQ']


def fundo_cnpj(df: pd.DataFrame, cnpj: str) -> DataFrame:
    """
    Separa o df do fundo de investimentos em várias categorias.

    Parameters:
    df: Dataframe que contém os ativos dos fundos.
    cnpj: cnpj do fundo de investimento que você está procurando.

    Returns:
    Vários dataframes de categorias diferentes: ações, BDRs, investimentos no exterior, cotas de fundos e títulos públicos.
    """
    # Lendo o df concatenado
    filt_cnpj = df['CNPJ_FUNDO'] == cnpj
    fundo_espec = df.loc[filt_cnpj]

    # Ações
    filt_acoes = (fundo_espec['TP_APLIC'] == 'Ações')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_acoes = fundo_espec.loc[filt_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_acao = lambda x: (x / df_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_acoes['PORCENTAGEM'] = list(map(porcentagem_acao, df_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_acoes = df_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # BDRs
    filt_bdr = (fundo_espec['TP_APLIC'] == 'Brazilian Depository Receipt - BDR')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_bdr = fundo_espec.loc[filt_bdr].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_bdr = lambda x: (x / df_bdr['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_bdr['PORCENTAGEM'] = list(map(porcentagem_bdr, df_bdr['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_bdr =  df_bdr.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Investimentos Exterior
    filt_exterior = (fundo_espec['TP_APLIC'] == 'Investimento no Exterior')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_exterior = fundo_espec.loc[filt_exterior].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_exterior = lambda x: (x / df_exterior['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_exterior['PORCENTAGEM'] = list(map(porcentagem_exterior, df_exterior['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_exterior = df_exterior.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Cotas de Fundos
    filt_cotas_fundos = (fundo_espec['TP_APLIC'] == 'Cotas de Fundos')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_cotas_fundos = fundo_espec.loc[filt_cotas_fundos].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada cota de fundo
    porcentagem_cotas = lambda x: (x / df_cotas_fundos['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_cotas_fundos['PORCENTAGEM'] = list(map(porcentagem_cotas, df_cotas_fundos['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_cotas_fundos = df_cotas_fundos.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Títulos públicos
    filt_titulos_pub = (fundo_espec['TP_APLIC'] == 'Títulos Públicos')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_titulos_pub = fundo_espec.loc[filt_titulos_pub].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada título público
    porcentagem_titulos = lambda x: (x / df_titulos_pub['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_titulos_pub['PORCENTAGEM'] = list(map(porcentagem_titulos, df_titulos_pub['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_titulos_pub = df_titulos_pub.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Obrigações por ações e outros TVM recebidos em empréstimo
    filt_vendido_acoes = (fundo_espec['TP_APLIC'] == 'Obrigações por ações e outros TVM recebidos em empréstimo')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_vendido_acoes = fundo_espec.loc[filt_vendido_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada título público
    porcentagem_vendido = lambda x: (x / df_vendido_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_vendido_acoes['PORCENTAGEM'] = list(map(porcentagem_vendido, df_vendido_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_vendido_acoes = df_vendido_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    return (
        df_acoes, 
        df_bdr, 
        df_exterior, 
        df_cotas_fundos, 
        df_titulos_pub, 
        df_vendido_acoes
    )


def fundo_cnpj_acoes(df: pd.DataFrame, cnpj: str) -> DataFrame:
    """
    Separa o df do fundo de investimento apenas na categoria de ações.

    Parameters:
    df: Dataframe que contém os ativos dos fundos.
    cnpj: cnpj do fundo de investimento que você está procurando.

    Returns:
    Vários dataframes de categorias diferentes: ações, BDRs, investimentos no exterior, cotas de fundos e títulos públicos.
    
    """
    # Lendo o df concatenado
    filt_cnpj = df['CNPJ_FUNDO'] == cnpj
    fundo_espec = df.loc[filt_cnpj]

    # Ações
    filt_acoes = (fundo_espec['TP_APLIC'] == 'Ações')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_acoes = fundo_espec.loc[filt_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_acao = lambda x: (x / df_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_acoes['PORCENTAGEM'] = list(map(porcentagem_acao, df_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_acoes = df_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    return df_acoes


def comparar_portfolios(df: pd.DataFrame, nome_fundo: str) -> str:
    """
    Comparação do portfólio - quais foram as ações que foram compradas e vendidas em relação ao mês anteirior.

    Pameters:
    df: DataFrame de cada mês do portfólio do fundo.
    nome_fundo: nome do fundo.

    Returns:
    Texto com as mudanças do portfólio.
    """
    for i in range(1, len(df)):
        mes_atual = df.iloc[i]
        mes_anterior = df.iloc[i - 1]
        
        data_atual = mes_atual['data']
        data_anterior = mes_anterior['data']
        
        acoes_atual = mes_atual['CD_ATIVO']
        acoes_anterior = mes_anterior['CD_ATIVO']
        
        vendidas = acoes_anterior - acoes_atual
        compradas = acoes_atual - acoes_anterior
        
        print(f'Comparando o portfólio de {data_anterior.strftime("%m/%Y")} e {data_atual.strftime("%m/%Y")}:')
        print(f'O {nome_fundo} vendeu as ações: {vendidas}')
        print(f'O {nome_fundo} comprou as ações: {compradas}')
        print('-' * 80)


def num_total_acoes(df: pd.DataFrame) -> pd.Series:
    """
    Parameters:
    df: DataFrame do portfólio do fundo.

    Returns:
    Número total de ações de cada mês do portfpolio.
    """
    num_total_acoes = df.groupby('data')['CD_ATIVO'].count()
    
    return num_total_acoes


def rank_top_5(df: pd.DataFrame) -> pd.Series:
    """
    Parameters:
    df: DataFrame do portfólio do fundo.
    
    Returns:
    rank_portfolio_fundo: rank das 5 maiores posições do fundo.
    """
    # Selecionando os valores únicos (datas) do df
    lst_data = df.index.unique()

    # Rank das 5 maiores posições do fundo
    lst_rank = []
    for _ in lst_data:
        rank = df.loc[_].nlargest(5, 'PORCENTAGEM')[['CD_ATIVO', 'PORCENTAGEM']]
        lst_rank.append(rank)

    # Concatendo os dfs de rank
    rank_portfolio = pd.concat(lst_rank)
    rank_portfolio_fundo = rank_portfolio.groupby('data')['CD_ATIVO'].apply(list)

    return rank_portfolio_fundo


def plot_portfolio(df: pd.DataFrame, nome_fundo: str):
    """
    Parameters:
    df: Dataframe do portfólio do fundo selecionado.
    nome_fundo: nome do fundo.

    Returns:
    Plot do portfólio do fundo.
    """
    # Extraindo os meses únicos do índice
    months = df.index.to_period('M').unique()

    # Títulos dos subplots
    titulos = [f'Distribuição Percentual do Portfólio - {month.strftime("%m/%Y")}' for month in months]

    # Criando a figura com subplots
    fig = make_subplots(rows=len(months),
                        cols=1,
                        subplot_titles=titulos,
                        vertical_spacing=0.02  # Espaço entre os plots
    )

    # Iterando sobre os meses
    for idx, month in enumerate(months):
        # Extraindo os dados do portfólio para o mês atual
        portfolio_mes = df.loc[month.strftime('%Y-%m'), ['CD_ATIVO', 'PORCENTAGEM']]
        
        fig.add_trace(go.Bar(
            x=portfolio_mes['PORCENTAGEM'] * 100,
            y=portfolio_mes['CD_ATIVO'],
            orientation='h',
            name=month.strftime('%m/%Y')
        ), row=idx + 1, col=1)

    fig.update_layout(
        title=f'Portfólio do Fundo {nome_fundo}',
        height=6000,
        width=900
    )

    return fig.show()

In [ ]:
# Formatando os arquivos parquet 'cda_fi'
df_cda_1 = open_cda_1(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_1_{ano}{mes}.parquet'
)
df_cda_2 = open_cda_2(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_2_{ano}{mes}.parquet'
)
df_cda_4 = open_cda_4(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_4_{ano}{mes}.parquet'
)
df_cda_7 = open_cda_7(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_7_{ano}{mes}.parquet'
)
df_cda_8 = open_cda_8(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_8_{ano}{mes}.parquet'
)

# Concatenando os dfs
df_ativos = pd.concat([df_cda_1, df_cda_2, df_cda_4, df_cda_7, df_cda_8])
df_ativos.tail()

In [3]:
# Formatando os arquivos parquet 'inf_diario_fi'

# Caminho da pasta dos arquivos 'inf_diario_fi'
path_inf_diario = 'C:\\Users\\vitor\\projetos_python\\python_b3\\historico-arquivos\\fundos_investimentos\\inf_diario_cvm\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos = glob.glob(path_inf_diario + '*.parquet')

# Lendo os arquivos parquet e juntando em apenas um df
lst_df = [pd.read_parquet(path) for path in paths_arquivos]

# Concatenando os dfs
df_inf_diario = pd.concat(lst_df, ignore_index=True)

# Selecionando apenas os 'Fundos de Investimentos'
filt_fi = df_inf_diario['TP_FUNDO'] == 'FI'
df_inf_diario = df_inf_diario.loc[filt_fi]

# Selecionando as principais colunas
df_inf_diario = df_inf_diario[['TP_FUNDO', 'CNPJ_FUNDO', 'DT_COMPTC' , 'VL_TOTAL', 'VL_QUOTA', 'VL_PATRIM_LIQ']]

# Transformando os dtypes das colunas.
df_inf_diario['TP_FUNDO'] = df_inf_diario['TP_FUNDO'].astype(str)
df_inf_diario['CNPJ_FUNDO'] = df_inf_diario['CNPJ_FUNDO'].astype(str)
df_inf_diario['DT_COMPTC'] = pd.to_datetime(df_inf_diario['DT_COMPTC'], format='%Y-%m-%d')

# Transformando a coluna 'DT_COMPTC' em index
df_inf_diario = df_inf_diario.set_index('DT_COMPTC')
df_inf_diario.tail()

,TP_FUNDO,CNPJ_FUNDO,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ
DT_COMPTC,,,,,
2024-04-24,FI,97.929.213/0001-34,84757562.92,11.771143,84612563.16
2024-04-25,FI,97.929.213/0001-34,84725908.95,11.766658,84580324.04
2024-04-26,FI,97.929.213/0001-34,84999412.47,11.804625,84853242.58
2024-04-29,FI,97.929.213/0001-34,85055425.98,11.812336,84908669.75
2024-04-30,FI,97.929.213/0001-34,84742750.95,11.768756,84595408.13


In [4]:
# IBOVESPA
ibov = yf.download('^BVSP', start='2023-01-01')[['Open', 'Adj Close']]

# Calculando a rentabilidade mensal do benchmark 
rentabilidade_mensal = {}
for date, group in ibov.groupby(ibov.index.to_period('M')):
    open_price_first_day = group['Open'].iloc[0]
    adj_close_last_day = group['Adj Close'].iloc[-1]
    rentabilidade = ((adj_close_last_day / open_price_first_day) - 1) * 100
    rentabilidade_mensal[date.to_timestamp()] = round(rentabilidade, 2)

# Transformando o dicionário em um df
df_ret_ibov_mensal = pd.DataFrame(list(rentabilidade_mensal.items()), columns=['data', 'ret_ibov'])
# Transformando a coluna 'data' em index
df_ret_ibov_mensal = df_ret_ibov_mensal.set_index('data')
# Transformando o formato do index ('ano-mes') p/ conseguir concatenar com as rentabilidades dos fundos
df_ret_ibov_mensal.index = df_ret_ibov_mensal.index.to_period('M')


# Calculando a rentabilidade anual do benchmark 
rentabilidade_anual = {}
for date, group in ibov.groupby(ibov.index.to_period('Y')):
    open_price_first_day = group['Open'].iloc[0]
    adj_close_last_day = group['Adj Close'].iloc[-1]
    rentabilidade = ((adj_close_last_day / open_price_first_day) - 1) * 100
    rentabilidade_anual[date.to_timestamp()] = round(rentabilidade, 2)

# Transformando o dicionário em um df
df_ret_ibov_anual = pd.DataFrame(list(rentabilidade_anual.items()), columns=['data', 'ret_ibov'])
# Transformando a coluna 'data' em index
df_ret_ibov_anual = df_ret_ibov_anual.set_index('data')
# Transformando o formato do index ('ano-mes') p/ conseguir concatenar com as rentabilidades dos fundos
df_ret_ibov_anual.index = df_ret_ibov_anual.index.to_period('Y')

print(df_ret_ibov_mensal)
print(df_ret_ibov_anual)

[*********************100%***********************]  1 of 1 completed
         ret_ibov
data             
2023-01      3.46
2023-02     -7.49
2023-03     -2.91
2023-04      2.50
2023-05      3.74
2023-06      9.00
2023-07      3.26
2023-08     -5.09
2023-09      0.71
2023-10     -2.93
2023-11     12.53
2023-12      5.38
2024-01     -4.79
2024-02      0.99
2024-03     -0.71
2024-04     -1.70
2024-05     -3.04
      ret_ibov
data          
2023     22.28
2024     -9.01


In [ ]:
# Quais são os tipos de ativos o fundo de investimento possui
lista_ativos = df_ativos['TP_APLIC'].drop_duplicates().to_list()
lista_ativos

## Fundos de Investimentos

### Fundo Verde

* https://maisretorno.com/fundo/verde-am-acoes-master-fia-1

In [5]:
# Rentabilidade mensal 
df_ret_mensal_verde = rentabilidade_fundo(df=df_inf_diario, cnpj='16.929.553/0001-63', nome_fundo='verde')[0]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_mensal_verde_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_mensal_verde,
    df_benchmark=df_ret_ibov_mensal,
    nome_fundo='verde',
    nome_benchmark='ibov'
)

df_ret_mensal_verde_ibov

,ret_verde,ret_ibov,performance
2023-01,0.65,3.46,-2.81
2023-02,-4.23,-7.49,3.26
2023-03,-1.75,-2.91,1.16
2023-04,-1.48,2.50,-3.98
2023-05,5.05,3.74,1.31
2023-06,7.85,9.00,-1.15
2023-07,2.09,3.26,-1.17
2023-08,-2.25,-5.09,2.84
2023-09,-0.18,0.71,-0.89
2023-10,-6.01,-2.93,-3.08


In [6]:
# Rentabilidade anual
df_ret_anual_verde = rentabilidade_fundo(df=df_inf_diario, cnpj='16.929.553/0001-63', nome_fundo='verde')[1]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_anual_verde_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_anual_verde,
    df_benchmark=df_ret_ibov_anual,
    nome_fundo='verde',
    nome_benchmark='ibov'
)

df_ret_anual_verde_ibov

,ret_verde,ret_ibov,performance
2023,11.57,22.28,-10.71
2024,-10.44,-9.01,-1.43


In [ ]:
verde_acoes, verde_bdr, verde_exterior, verde_cotas_fundos, verde_titulos_pub, verde_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='16.929.553/0001-63')
verde_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='16.929.553/0001-63'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(verde_acoes['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_bdr = round(pd.Series(verde_bdr['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_exterior = round(pd.Series(verde_exterior['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(verde_cotas_fundos['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_titulos_pub = round(pd.Series(verde_titulos_pub['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(verde_vendido_acoes['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
verde_acoes = pd.concat([verde_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
verde_acoes['data'] = data
# Definindo a coluna 'data' como o index
verde_acoes = verde_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
verde_bdr = pd.concat([verde_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
verde_bdr['data'] = data
# Definindo a coluna 'data' como o index
verde_bdr = verde_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
verde_exterior = pd.concat([verde_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
verde_exterior['data'] = data
# Definindo a coluna 'data' como o index
verde_exterior = verde_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
verde_cotas_fundos = pd.concat([verde_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
verde_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
verde_cotas_fundos = verde_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
verde_titulos_pub = pd.concat([verde_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
verde_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
verde_titulos_pub = verde_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
verde_vendido_acoes = pd.concat([verde_vendido_acoes, df_linha_pct_vendido_acoes ])
# Criando a coluna 'data' no df
verde_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
verde_veverde_vendido_acoes =verde_vendido_acoes.set_index('data')

In [ ]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//verde//fundo_verde_{ano}{mes}.xlsx') as writer:
    verde_acoes.to_excel(writer, sheet_name='acoes')
    verde_bdr.to_excel(writer, sheet_name='bdr')
    verde_exterior.to_excel(writer, sheet_name='exterior')
    verde_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    verde_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    verde_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [ ]:
# Caminho da pasta do fundo Verde
path_verde = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\arquivos_fundos\\verde\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_verde = glob.glob(path_verde + '*.xlsx')

# Lendo cada arquivo excel
lst_dfs = []
for path in paths_arquivos_verde:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_verde = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_verde = portfolio_verde[portfolio_verde['VL_MERC_POS_FINAL'] != 0]
portfolio_verde = portfolio_verde.dropna()

# Definindo a coluna 'data' como o index
portfolio_verde = portfolio_verde.set_index('data')
portfolio_verde

In [ ]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_verde.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Verde')

In [ ]:
# Número total de ações de cada mês do portfpolio
num_total_acoes(df=portfolio_verde)

In [ ]:
# Rank das 5 maiores ações do fundo
rank_top_5(df=portfolio_verde)

In [ ]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_verde, nome_fundo='Verde')

### Fundo Dynamo

* https://maisretorno.com/fundo/dynamo-cougar-master-fia

In [8]:
# Rentabilidade mensal
df_ret_mensal_dynamo = rentabilidade_fundo(df=df_inf_diario, cnpj='37.916.879/0001-26', nome_fundo='dynamo')[0]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_mensal_dynamo_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_mensal_dynamo,
    df_benchmark=df_ret_ibov_mensal,
    nome_fundo='dynamo',
    nome_benchmark='ibov'
)

df_ret_mensal_dynamo_ibov

,ret_dynamo,ret_ibov,performance
2023-01,4.80,3.46,1.34
2023-02,-4.66,-7.49,2.83
2023-03,-6.02,-2.91,-3.11
2023-04,-0.24,2.50,-2.74
2023-05,12.20,3.74,8.46
2023-06,8.40,9.00,-0.60
2023-07,5.00,3.26,1.74
2023-08,-6.35,-5.09,-1.26
2023-09,-0.77,0.71,-1.48
2023-10,-4.90,-2.93,-1.97


In [9]:
# Rentabilidade anual
df_ret_anual_dynamo = rentabilidade_fundo(df=df_inf_diario, cnpj='37.916.879/0001-26', nome_fundo='dynamo')[1]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_anual_dynamo_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_anual_dynamo,
    df_benchmark=df_ret_ibov_anual,
    nome_fundo='dynamo',
    nome_benchmark='ibov'
)

df_ret_anual_dynamo_ibov

,ret_dynamo,ret_ibov,performance
2023,24.92,22.28,2.64
2024,-2.81,-9.01,6.20


In [ ]:
dynamo_acoes, dynamo_bdr, dynamo_exterior, dynamo_cotas_fundos, dynamo_titulos_pub, dynamo_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='37.916.879/0001-26')
dynamo_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet',
    cnpj='37.916.879/0001-26'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(dynamo_acoes['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_bdr = round(pd.Series(dynamo_bdr['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_exterior = round(pd.Series(dynamo_exterior['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(dynamo_cotas_fundos['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_titulos_pub = round(pd.Series(dynamo_titulos_pub['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(dynamo_vendido_acoes['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
dynamo_acoes = pd.concat([dynamo_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
dynamo_acoes['data'] = data
# Definindo a coluna 'data' como o index
dynamo_acoes = dynamo_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
dynamo_bdr = pd.concat([dynamo_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
dynamo_bdr['data'] = data
# Definindo a coluna 'data' como o index
dynamo_bdr = dynamo_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
dynamo_exterior = pd.concat([dynamo_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
dynamo_exterior['data'] = data
# Definindo a coluna 'data' como o index
dynamo_exterior = dynamo_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
dynamo_cotas_fundos = pd.concat([dynamo_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
dynamo_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
dynamo_codynamo_cotas_fundos =dynamo_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
dynamo_titulos_pub = pd.concat([dynamo_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
dynamo_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
dynamo_titulos_pub = dynamo_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
dynamo_vendido_acoes = pd.concat([dynamo_vendido_acoes, df_linha_pct_vendido_acoes])
# Criando a coluna 'data' no df
dynamo_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
dynamo_vendido_acoes = dynamo_vendido_acoes.set_index('data')

In [ ]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//dynamo//fundo_dynamo_{ano}{mes}.xlsx') as writer:
    dynamo_acoes.to_excel(writer, sheet_name='acoes')
    dynamo_bdr.to_excel(writer, sheet_name='bdr')
    dynamo_exterior.to_excel(writer, sheet_name='exterior')
    dynamo_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    dynamo_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    dynamo_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [ ]:
# Caminho da pasta do fundo Dynamo
path_dynamo = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\arquivos_fundos\\dynamo\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_dynamo = glob.glob(path_dynamo + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_dynamo:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_dynamo = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_dynamo = portfolio_dynamo[portfolio_dynamo['VL_MERC_POS_FINAL'] != 0]
portfolio_dynamo = portfolio_dynamo.dropna()

# Definindo a coluna 'data' como o index
portfolio_dynamo = portfolio_dynamo.set_index('data')
portfolio_dynamo

In [ ]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_dynamo.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Dynamo')

In [ ]:
# Número total de ações de cada mês do portfpolio
num_total_acoes(df=portfolio_dynamo)

In [ ]:
# Rank das 5 maiores ações do fundo
rank_top_5(df=portfolio_dynamo)

In [ ]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_dynamo, nome_fundo='Dynamo')

### Fundo IP

* https://maisretorno.com/fundo/ip-participacoes-master-fia-bdr-nivel-i

In [10]:
# Rentabilidade mensal
df_ret_mensal_ip = rentabilidade_fundo(df=df_inf_diario, cnpj='11.435.298/0001-89', nome_fundo='ip')[0]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_mensal_ip_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_mensal_ip,
    df_benchmark=df_ret_ibov_mensal,
    nome_fundo='ip',
    nome_benchmark='ibov'
)

df_ret_mensal_ip_ibov

,ret_ip,ret_ibov,performance
2023-01,5.11,3.46,1.65
2023-02,-5.15,-7.49,2.34
2023-03,0.56,-2.91,3.47
2023-04,3.51,2.50,1.01
2023-05,3.03,3.74,-0.71
2023-06,7.54,9.00,-1.46
2023-07,4.21,3.26,0.95
2023-08,-0.94,-5.09,4.15
2023-09,-1.87,0.71,-2.58
2023-10,-3.69,-2.93,-0.76


In [11]:
# Rentabilidade anual
df_ret_anual_ip = rentabilidade_fundo(df=df_inf_diario, cnpj='11.435.298/0001-89', nome_fundo='ip')[1]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_anual_ip_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_anual_ip,
    df_benchmark=df_ret_ibov_anual,
    nome_fundo='ip',
    nome_benchmark='ibov'
)

df_ret_anual_ip_ibov

,ret_ip,ret_ibov,performance
2023,30.94,22.28,8.66
2024,1.72,-9.01,10.73


In [ ]:
ip_acoes, ip_bdr, ip_exterior, ip_cotas_fundos, ip_titulos_pub, ip_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='11.435.298/0001-89')
ip_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='11.435.298/0001-89'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(ip_acoes['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_bdr = round(pd.Series(ip_bdr['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_exterior = round(pd.Series(ip_exterior['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(ip_cotas_fundos['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_titulos_pub = round(pd.Series(ip_titulos_pub['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(ip_vendido_acoes['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
ip_acoes = pd.concat([ip_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
ip_acoes['data'] = data
# Definindo a coluna 'data' como o index
ip_acoes = ip_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
ip_bdr = pd.concat([ip_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
ip_bdr['data'] = data
# Definindo a coluna 'data' como o index
ip_bdr = ip_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
ip_exterior = pd.concat([ip_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
ip_exterior['data'] = data
# Definindo a coluna 'data' como o index
ip_exterior = ip_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
ip_cotas_fundos = pd.concat([ip_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
ip_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
ip_cotas_fundos = ip_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
ip_titulos_pub = pd.concat([ip_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
ip_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
ip_titulos_pub = ip_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
ip_vendido_acoes = pd.concat([ip_vendido_acoes, df_linha_pct_vendido_acoes])
# Criando a coluna 'data' no df
ip_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
ip_vendido_acoes = ip_vendido_acoes.set_index('data')

In [ ]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//ip//fundo_ip_{ano}{mes}.xlsx') as writer:
    ip_acoes.to_excel(writer, sheet_name='acoes')
    ip_bdr.to_excel(writer, sheet_name='bdr')
    ip_exterior.to_excel(writer, sheet_name='exterior')
    ip_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    ip_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    ip_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [ ]:
# Caminho da pasta do fundo IP
path_ip = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\arquivos_fundos\\ip\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_ip = glob.glob(path_ip + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_ip:
    df = pd.read_excel(path, sheet_name='bdr')
    lst_dfs.append(df)

# Concatendo os dfs de bdrs
portfolio_ip = pd.concat(lst_dfs, ignore_index=True)

# Retirando as bdrs que não tem nenhum dinheiro alocado e os NAN
portfolio_ip = portfolio_ip[portfolio_ip['VL_MERC_POS_FINAL'] != 0]
portfolio_ip = portfolio_ip.dropna()

# Definindo a coluna 'data' como o index
portfolio_ip = portfolio_ip.set_index('data')
portfolio_ip

In [ ]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_ip.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='IP')

In [ ]:
# Número total de ações de cada mês do portfpolio
num_total_acoes(df=portfolio_ip)

In [ ]:
# Rank das 5 maiores ações do fundo
rank_top_5(df=portfolio_ip)

In [ ]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_ip, nome_fundo='IP')

### Fundo Squadra

* https://maisretorno.com/fundo/squadra-master-long-biased-fia
* Famosos pelos shorts de:
    * IRBR - 05/2018
    * AERI3 - 01/2021
    * Nubank

In [12]:
# Rentabilidade mensal
df_ret_mensal_squadra = rentabilidade_fundo(df=df_inf_diario, cnpj='09.412.648/0001-40', nome_fundo='squadra')[0]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_mensal_squadra_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_mensal_squadra,
    df_benchmark=df_ret_ibov_mensal,
    nome_fundo='squadra',
    nome_benchmark='ibov'
)

df_ret_mensal_squadra_ibov

,ret_squadra,ret_ibov,performance
2023-01,5.23,3.46,1.77
2023-02,-6.62,-7.49,0.87
2023-03,-0.20,-2.91,2.71
2023-04,3.92,2.50,1.42
2023-05,8.83,3.74,5.09
2023-06,8.13,9.00,-0.87
2023-07,4.00,3.26,0.74
2023-08,2.24,-5.09,7.33
2023-09,1.13,0.71,0.42
2023-10,-0.69,-2.93,2.24


In [13]:
# Rentabilidade anual
df_ret_anual_squadra = rentabilidade_fundo(df=df_inf_diario, cnpj='09.412.648/0001-40', nome_fundo='squadra')[1]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_anual_squadra_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_anual_squadra,
    df_benchmark=df_ret_ibov_anual,
    nome_fundo='squadra',
    nome_benchmark='ibov'
)

df_ret_anual_squadra_ibov

,ret_squadra,ret_ibov,performance
2023,44.86,22.28,22.58
2024,-3.84,-9.01,5.17


In [ ]:
squadra_acoes, squadra_bdr, squadra_exterior, squadra_cotas_fundos, squadra_titulos_pub, squadra_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='09.412.648/0001-40')
squadra_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='09.412.648/0001-40'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(squadra_acoes['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_bdr = round(pd.Series(squadra_bdr['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_exterior = round(pd.Series(squadra_exterior['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(squadra_cotas_fundos['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_titulos_pub = round(pd.Series(squadra_titulos_pub['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(squadra_vendido_acoes['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
squadra_acoes = pd.concat([squadra_acoes, df_linha_pct_acoes], ignore_index=True)
# Criando a coluna 'data' no df
squadra_acoes['data'] = data
# Definindo a coluna 'data' como o index
squadra_acoes = squadra_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
squadra_bdr = pd.concat([squadra_bdr, df_linha_pct_bdr], ignore_index=True)
# Criando a coluna 'data' no df
squadra_bdr['data'] = data
# Definindo a coluna 'data' como o index
squadra_bdr = squadra_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
squadra_exterior = pd.concat([squadra_exterior, df_linha_pct_exterior], ignore_index=True)
# Criando a coluna 'data' no df
squadra_exterior['data'] = data
# Definindo a coluna 'data' como o index
squadra_exterior = squadra_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
squadra_cotas_fundos = pd.concat([squadra_cotas_fundos, df_linha_pct_cotas_fundos], ignore_index=True)
# Criando a coluna 'data' no df
squadra_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
squadra_cotas_fundos = squadra_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
squadra_titulos_pub = pd.concat([squadra_titulos_pub, df_linha_pct_titulos_pub], ignore_index=True)
# Criando a coluna 'data' no df
squadra_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
squadra_titulos_pub = squadra_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
squadra_vendido_acoes = pd.concat([squadra_vendido_acoes, df_linha_pct_vendido_acoes], ignore_index=True)
# Criando a coluna 'data' no df
squadra_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
squadra_vendido_acoes = squadra_vendido_acoes.set_index('data')

In [ ]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//squadra//fundo_squadra_{ano}{mes}.xlsx') as writer:
    squadra_acoes.to_excel(writer, sheet_name='acoes')
    squadra_bdr.to_excel(writer, sheet_name='bdr')
    squadra_exterior.to_excel(writer, sheet_name='exterior')
    squadra_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    squadra_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    squadra_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [ ]:
# Caminho da pasta do fundo Squadra
path_squadra = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\arquivos_fundos\\squadra\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_squadra = glob.glob(path_squadra + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_squadra:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_squadra = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_squadra = portfolio_squadra[portfolio_squadra['VL_MERC_POS_FINAL'] != 0]
portfolio_squadra = portfolio_squadra.dropna()

# Definindo a coluna 'data' como o index
portfolio_squadra = portfolio_squadra.set_index('data')
portfolio_squadra

In [ ]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_squadra.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Squadra')

In [ ]:
# Número total de ações de cada mês do portfólio
num_total_acoes(df=portfolio_squadra)

In [ ]:
# Rank das 5 maiores ações do fundo
rank_top_5(df=portfolio_squadra)

In [ ]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_squadra, nome_fundo='Squadra')

### Fundo Guepardo

* https://maisretorno.com/fundo/guepardo-institucional-master-fia

In [14]:
# Rentabilidade mensal
df_ret_mensal_guepardo = rentabilidade_fundo(df=df_inf_diario, cnpj='14.213.077/0001-54', nome_fundo='guepardo')[0]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_mensal_guepardo_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_mensal_guepardo,
    df_benchmark=df_ret_ibov_mensal,
    nome_fundo='guepardo',
    nome_benchmark='ibov'
)

df_ret_mensal_guepardo_ibov

,ret_guepardo,ret_ibov,performance
2023-01,-0.74,3.46,-4.20
2023-02,-3.55,-7.49,3.94
2023-03,-0.86,-2.91,2.05
2023-04,6.05,2.50,3.55
2023-05,14.29,3.74,10.55
2023-06,9.84,9.00,0.84
2023-07,5.92,3.26,2.66
2023-08,-2.77,-5.09,2.32
2023-09,1.84,0.71,1.13
2023-10,-3.34,-2.93,-0.41


In [15]:
# Rentabilidade anual
df_ret_anual_guepardo = rentabilidade_fundo(df=df_inf_diario, cnpj='14.213.077/0001-54', nome_fundo='guepardo')[1]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_anual_guepardo_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_anual_guepardo,
    df_benchmark=df_ret_ibov_anual,
    nome_fundo='guepardo',
    nome_benchmark='ibov'
)

df_ret_anual_guepardo_ibov

,ret_guepardo,ret_ibov,performance
2023,49.26,22.28,26.98
2024,-3.91,-9.01,5.10


In [ ]:
guepardo_acoes = fundo_cnpj_acoes(df=df_ativos, cnpj='14.213.077/0001-54')
guepardo_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='14.213.077/0001-54'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(guepardo_acoes['VL_MERC_POS_FINAL'].sum() / guepardo_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
guepardo_acoes = pd.concat([guepardo_acoes, df_linha_pct_acoes], ignore_index=True)
# Criando a coluna 'data' no df
guepardo_acoes['data'] = data
# Definindo a coluna 'data' como o index
guepardo_acoes = guepardo_acoes.set_index('data')

In [ ]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//guepardo//fundo_guepardo_{ano}{mes}.xlsx') as writer:
    guepardo_acoes.to_excel(writer, sheet_name='acoes')
    

In [ ]:
# Caminho da pasta do fundo Guepardo
path_guepardo = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\arquivos_fundos\\guepardo\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_guepardo = glob.glob(path_guepardo + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_guepardo:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_guepardo = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado  e os NAN
portfolio_guepardo = portfolio_guepardo[portfolio_guepardo['VL_MERC_POS_FINAL'] != 0]
portfolio_guepardo = portfolio_guepardo.dropna()

# Definindo a coluna 'data' como o index
portfolio_guepardo = portfolio_guepardo.set_index('data')
portfolio_guepardo

In [ ]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_guepardo.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Guepardo')

In [ ]:
# Número total de ações de cada mês do portfpolio
num_total_acoes(df=portfolio_guepardo)

In [ ]:
# Rank das 5 maiores ações do fundo
rank_top_5(df=portfolio_guepardo)

In [ ]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_guepardo, nome_fundo='Guepardo')